In [3]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
import pandas as pd
# Import the patch library
from pathlib import Path

# Create a Path object that points to the directory with data
corpus_dir = Path("5charismatic.txt")
text = corpus_dir.read_text(encoding='utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [4]:
movie_data = load_files(r"txt_sentoken")
X, y = movie_data.data, movie_data.target

In [6]:
nltk.download('wordnet')

documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...


In [7]:
#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
#X = vectorizer.fit_transform(documents).toarray()

In [8]:
#from sklearn.feature_extraction.text import TfidfTransformer
#tfidfconverter = TfidfTransformer()
#X = tfidfconverter.fit_transform(X).toarray()

In [ ]:
# More direct way to execute previous 2 cells as 1
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

In [9]:
# divides data into 20% test set and 80% training set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
from sklearn.ensemble import RandomForestClassifier
# To train our machine learning model using the random forest algorithm we will 
# use RandomForestClassifier class from the sklearn.ensemble library. 
# The fit method of this class is used to train the algorithm.
# We need to pass the training data and training target sets to this method

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

# to predict the sentiment for the documents we can use the predict method
y_pred = classifier.predict(X_test)

Evaluating Model

In [12]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[180  28]
 [ 30 162]]
              precision    recall  f1-score   support

           1       0.86      0.87      0.86       208
           2       0.85      0.84      0.85       192

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

0.855


In [13]:
# For larger datasets it is recommended to save the model once it is trained

with open('text_classifier', 'wb') as picklefile:
    pickle.dump(classifier,picklefile)

In [14]:
# For loading the model 

with open('text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

In [15]:
# Run the code again for the loaded model to see if the results match
y_pred2 = model.predict(X_test)

print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2)) 

[[180  28]
 [ 30 162]]
              precision    recall  f1-score   support

           1       0.86      0.87      0.86       208
           2       0.85      0.84      0.85       192

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

0.855
